<font face="Times New Roman" >
<font size=3 >
Global lakes can be divided into different categories according to size area, chlorophyll a (Delsontro et al. 2018). For each size class and nutritional gradient (divided into oligotrophic, mesotrophic, eutrophic and hyper-eutrophic according to USA-EPA standards), we apply Monte Carlo analysis (Python_pymc3) (Deemer & Holgerson), 1000 iterations were performed for the size class of each lake. Considering the uncertainty of area and chlorophyll a, Monte Carlo analysis only includes sites with surface area information and chlorophyll a (n = 596 lakes). Since the data are skewed, we perform a logarithmic transformation before Monte Carlo analysis. Each iteration randomly selected a value (‘N_ Depletion’、‘P_ Depletion’、‘N_ Enrichment’、‘P _ Enrichment ’) from a normal distribution surrounding the mean and standard deviation for that size class. Simulations also selected for a surface area estimate of lakes in each size class using a uniform distribution based on estimates from reference (Downing, et al.2010). We then estimated the value (‘N_ Depletion’、‘P_ Depletion’、‘N_ Enrichment’、‘P _ Enrichment ’) by applying surface areas across 20 chlorophyll a bins (with each bin spanning 5 µg/L−1 from 0 to 100 µg/L −1 ), then calculating total values from each bin and finally summing across the 20 bins (see Delsontro et al. 2018 for the joint distribution table). The global distribution of chlorophyll a concentration was generated from MERIS OC4 satellite images through the method of Michigan Institute of technology, which is based on 300 m resolution input (Sayers et al. 2015).</font> 

<font face="Times New Roman" >
<font size=3 >
The calculation process is as follows

In [ ]:
# -*- coding: utf-8 -*-
"""
@author: WZ
"""
import pymc3 as pm
from sympy import symbols, solve
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as logitm
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
import statsmodels.regression.linear_model as lm_
import statsmodels.robust.robust_linear_model as roblm_
import statsmodels.regression.quantile_regression as QREG
from statsmodels.stats.outliers_influence import summary_table
import itertools
from sympy import symbols, solve
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as logitm
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
import math
import seaborn as sns
import statsmodels.regression.linear_model as lm_
import statsmodels.robust.robust_linear_model as roblm_
import statsmodels.regression.quantile_regression as QREG
import time
import math
from scipy.stats import pearsonr
from sklearn import preprocessing  
from rpy2.robjects.packages import importr
import rpy2.robjects as robjects
def get_average(records):
    return sum(records) / len(records)
def get_variance(records):
    average = get_average(records)
    return sum([(x - average) ** 2 for x in records]) / len(records)
def get_standard_deviation(records):
    variance = get_variance(records)
    return math.sqrt(variance)
def get_z_score(records):
    avg = get_average(records)
    stan = get_standard_deviation(records)
    scores = [(i-avg)/stan for i in records]
    return scores
def log_series(x):
    ou = []
    for ir in range(0,len(x)):
        try:
            xs = math.log(x[ir],math.e)
            ou.append(xs)
        except:
            xs = np.nan
            ou.append(xs)
    oui = pd.Series(ou)
    return oui 
def log_A(x):
    xs = math.log(x,math.e)
    return xs
#%% UPSCALING
filepathQ = r'C:\Users\JL\Desktop\File\data'
eEdata = pd.read_csv(filepathQ + '\\Lakbase_Data.csv')
upscaling = pd.read_csv(filepathQ + '\\Upscaling_distribution.csv')
disemission = pd.read_table(filepathQ + '\\Distri_base.txt',sep=',')
disemissionchla = pd.read_csv(filepathQ + '\\chladis_base.csv')
upscaling_database = pd.read_table(filepathQ +     "\\Hydro_ploy_database.txt",sep=',')


def Gusuan_Class(d1,d2):
    size4_a = eEdata[eEdata['Area'] >= d1]
    size4_b = size4_a[size4_a['Area'] < d2]
    size4 = size4_b
    size4_chla1 = size4[size4['Chla'] < 2]
    size4_chla2_a = size4[size4['Chla'] >= 2]
    size4_chla2 = size4_chla2_a[size4_chla2_a['Chla'] < 7 ]
    size4_chla3_a = size4[size4['Chla'] >= 7]
    size4_chla3 = size4_chla3_a[size4_chla3_a['Chla'] < 30 ]
    size4_chla4 = size4[size4['Chla'] >= 30]
    if len( size4_chla2) == 0:
        size4_chla2 = size4_chla1
    return size4_chla1,size4_chla2,size4_chla3,size4_chla4
def Gusuan_One(area,depth,size4_chla1,size4_chla2,size4_chla3,size4_chla4):
    SA = area
    value_of_Chla_1 = 0.1422 * SA
    value_of_Chla_2 = 0.3716 * SA
    value_of_Chla_3 = 0.3027 * SA
    value_of_Chla_4 = 0.1835 * SA
    er_N_dp1 = np.mean(size4_chla1['N_Depletion'])
    er_N_dp2 = np.mean(size4_chla2['N_Depletion'])
    er_N_dp3 = np.mean(size4_chla3['N_Depletion'])
    er_N_dp4 = np.mean(size4_chla4['N_Depletion'])
    er_P_dp1 = np.mean(size4_chla1['P_Depletion'])
    er_P_dp2 = np.mean(size4_chla2['P_Depletion'])
    er_P_dp3 = np.mean(size4_chla3['P_Depletion'])
    er_P_dp4 = np.mean(size4_chla4['P_Depletion'])
    er_N_en1 = np.mean(size4_chla1['N_Enrichment'])
    er_N_en2 = np.mean(size4_chla2['N_Enrichment'])
    er_N_en3 = np.mean(size4_chla3['N_Enrichment'])
    er_N_en4 = np.mean(size4_chla4['N_Enrichment'])
    er_P_en1 = np.mean(size4_chla1['P_Enrichment'])
    er_P_en2 = np.mean(size4_chla2['P_Enrichment'])
    er_P_en3 = np.mean(size4_chla3['P_Enrichment'])
    er_P_en4 = np.mean(size4_chla4['P_Enrichment'])
    ii_N_dp1 = (er_N_dp1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_N_dp2 = (er_N_dp2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_N_dp3 = (er_N_dp3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_N_dp4 = (er_N_dp4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_P_dp1 = (er_P_dp1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_P_dp2 = (er_P_dp2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_P_dp3 = (er_P_dp3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_P_dp4 = (er_P_dp4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_N_en1 = (er_N_en1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_N_en2 = (er_N_en2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_N_en3 = (er_N_en3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_N_en4 = (er_N_en4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_P_en1 = (er_P_en1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_P_en2 = (er_P_en2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_P_en3 = (er_P_en3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_P_en4 = (er_P_en4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ov_N_dp = ii_N_dp1 +  ii_N_dp2 +  ii_N_dp3 + ii_N_dp4
    ov_N_en = ii_N_en1 +  ii_N_en2 +  ii_N_en3 + ii_N_en4
    ov_P_dp = ii_P_dp1 +  ii_P_dp2 +  ii_P_dp3 + ii_P_dp4
    ov_P_en = ii_P_en1 +  ii_P_en2 +  ii_P_en3 + ii_P_en4
    return ov_N_dp,ov_P_dp,ov_N_en,ov_P_en
def MCMC(data):
    mu_mean = np.mean(data)
    return mu_mean
def MCMC2(data):
    mu_mean = pm.Normal("mu_a", mu=0.0, sigma=1)
    sigmas = pm.HalfNormal("sigma_a", 5.0)
    with pm.Model():
        res = pm.Normal("res", mu= mu_mean, sigma=sigmas, observed=data)
        trace= pm.sample(2000, tune=2000, target_accept=0.9, return_inferencedata=True)  ## Return the values of "mu_mean"
    return mu_mean

def Gusuan_TWO(area,depth,size4_chla1,size4_chla2,size4_chla3,size4_chla4):
    SA = area
    value_of_Chla_1 = 0.1422 * SA
    value_of_Chla_2 = 0.3716 * SA
    value_of_Chla_3 = 0.3027 * SA
    value_of_Chla_4 = 0.1835 * SA
    er_N_dp1 = MCMC(size4_chla1['N_Depletion'])
    er_N_dp2= MCMC(size4_chla2['N_Depletion'])
    er_N_dp3 = MCMC(size4_chla3['N_Depletion'])
    er_N_dp4 = MCMC(size4_chla4['N_Depletion'])
    er_P_dp1 = MCMC(size4_chla1['P_Depletion'])
    er_P_dp2= MCMC(size4_chla2['P_Depletion'])
    er_P_dp3= MCMC(size4_chla3['P_Depletion'])
    er_P_dp4 = MCMC(size4_chla4['P_Depletion'])
    er_N_en1 = MCMC(size4_chla1['N_Enrichment'])
    er_N_en2 = MCMC(size4_chla2['N_Enrichment'])
    er_N_en3 = MCMC(size4_chla3['N_Enrichment'])
    er_N_en4= MCMC(size4_chla4['N_Enrichment'])
    er_P_en1 = MCMC(size4_chla1['P_Enrichment'])
    er_P_en2 = MCMC(size4_chla2['P_Enrichment'])
    er_P_en3 = MCMC(size4_chla3['P_Enrichment'])
    er_P_en4 = MCMC(size4_chla4['P_Enrichment'])
    ii_N_dp1 = (er_N_dp1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_N_dp2 = (er_N_dp2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_N_dp3 = (er_N_dp3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_N_dp4 = (er_N_dp4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_P_dp1 = (er_P_dp1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_P_dp2 = (er_P_dp2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_P_dp3 = (er_P_dp3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_P_dp4 = (er_P_dp4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_N_en1 = (er_N_en1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_N_en2 = (er_N_en2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_N_en3 = (er_N_en3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_N_en4 = (er_N_en4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_P_en1 = (er_P_en1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_P_en2 = (er_P_en2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_P_en3 = (er_P_en3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_P_en4 = (er_P_en4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ov_N_dp = ii_N_dp1 +  ii_N_dp2 +  ii_N_dp3 + ii_N_dp4
    ov_N_en = ii_N_en1 +  ii_N_en2 +  ii_N_en3 + ii_N_en4
    ov_P_dp = ii_P_dp1 +  ii_P_dp2 +  ii_P_dp3 + ii_P_dp4
    ov_P_en = ii_P_en1 +  ii_P_en2 +  ii_P_en3 + ii_P_en4   
    return ov_N_dp,ov_P_dp,ov_N_en,ov_P_en
def Gusuan_plot(area,depth,size4_chla1,size4_chla2,size4_chla3,size4_chla4):
    SA = area
    value_of_Chla_1 = 0.1422 * SA
    value_of_Chla_2 = 0.3716 * SA
    value_of_Chla_3 = 0.3027 * SA
    value_of_Chla_4 = 0.1835 * SA
    er_N_dp1 = MCMC(size4_chla1['N_Depletion'])
    er_N_dp2= MCMC(size4_chla2['N_Depletion'])
    er_N_dp3 = MCMC(size4_chla3['N_Depletion'])
    er_N_dp4 = MCMC(size4_chla4['N_Depletion'])
    er_P_dp1 = MCMC(size4_chla1['P_Depletion'])
    er_P_dp2= MCMC(size4_chla2['P_Depletion'])
    er_P_dp3= MCMC(size4_chla3['P_Depletion'])
    er_P_dp4 = MCMC(size4_chla4['P_Depletion'])
    er_N_en1 = MCMC(size4_chla1['N_Enrichment'])
    er_N_en2 = MCMC(size4_chla2['N_Enrichment'])
    er_N_en3 = MCMC(size4_chla3['N_Enrichment'])
    er_N_en4= MCMC(size4_chla4['N_Enrichment'])
    er_P_en1 = MCMC(size4_chla1['P_Enrichment'])
    er_P_en2 = MCMC(size4_chla2['P_Enrichment'])
    er_P_en3 = MCMC(size4_chla3['P_Enrichment'])
    er_P_en4 = MCMC(size4_chla4['P_Enrichment'])
    ii_N_dp1 = (er_N_dp1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_N_dp2 = (er_N_dp2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_N_dp3 = (er_N_dp3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_N_dp4 = (er_N_dp4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_P_dp1 = (er_P_dp1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_P_dp2 = (er_P_dp2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_P_dp3 = (er_P_dp3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_P_dp4 = (er_P_dp4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_N_en1 = (er_N_en1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_N_en2 = (er_N_en2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_N_en3 = (er_N_en3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_N_en4 = (er_N_en4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_P_en1 = (er_P_en1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_P_en2 = (er_P_en2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_P_en3 = (er_P_en3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_P_en4 = (er_P_en4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ov_N_dp = ii_N_dp1 +  ii_N_dp2 +  ii_N_dp3 + ii_N_dp4
    ov_N_en = ii_N_en1 +  ii_N_en2 +  ii_N_en3 + ii_N_en4
    ov_P_dp = ii_P_dp1 +  ii_P_dp2 +  ii_P_dp3 + ii_P_dp4
    ov_P_en = ii_P_en1 +  ii_P_en2 +  ii_P_en3 + ii_P_en4   
    return ii_N_dp1,ii_N_dp2,ii_N_dp3,ii_N_dp4,ii_N_en1,ii_N_en2,ii_N_en3,ii_N_en4,ii_P_dp1,ii_P_dp2,ii_P_dp3,ii_P_dp4,ii_P_en1,ii_P_en2,ii_P_en3,ii_P_en4
def Gusuan_plot_area(area,depth,size4_chla1,size4_chla2,size4_chla3,size4_chla4):
    SA = area
    value_of_Chla_1 = 0.1422 * SA
    value_of_Chla_2 = 0.3716 * SA
    value_of_Chla_3 = 0.3027 * SA
    value_of_Chla_4 = 0.1835 * SA
    er_N_dp1 = MCMC(size4_chla1['N_Depletion'])
    er_N_dp2= MCMC(size4_chla2['N_Depletion'])
    er_N_dp3 = MCMC(size4_chla3['N_Depletion'])
    er_N_dp4 = MCMC(size4_chla4['N_Depletion'])
    er_P_dp1 = MCMC(size4_chla1['P_Depletion'])
    er_P_dp2= MCMC(size4_chla2['P_Depletion'])
    er_P_dp3= MCMC(size4_chla3['P_Depletion'])
    er_P_dp4 = MCMC(size4_chla4['P_Depletion'])
    er_N_en1 = MCMC(size4_chla1['N_Enrichment'])
    er_N_en2 = MCMC(size4_chla2['N_Enrichment'])
    er_N_en3 = MCMC(size4_chla3['N_Enrichment'])
    er_N_en4= MCMC(size4_chla4['N_Enrichment'])
    er_P_en1 = MCMC(size4_chla1['P_Enrichment'])
    er_P_en2 = MCMC(size4_chla2['P_Enrichment'])
    er_P_en3 = MCMC(size4_chla3['P_Enrichment'])
    er_P_en4 = MCMC(size4_chla4['P_Enrichment'])
    ii_N_dp1 = (er_N_dp1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_N_dp2 = (er_N_dp2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_N_dp3 = (er_N_dp3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_N_dp4 = (er_N_dp4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_P_dp1 = (er_P_dp1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_P_dp2 = (er_P_dp2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_P_dp3 = (er_P_dp3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_P_dp4 = (er_P_dp4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_N_en1 = (er_N_en1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_N_en2 = (er_N_en2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_N_en3 = (er_N_en3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_N_en4 = (er_N_en4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    ii_P_en1 = (er_P_en1*365*(value_of_Chla_1*1000000*depth))/1000000000000
    ii_P_en2 = (er_P_en2*365*(value_of_Chla_2*1000000*depth))/1000000000000
    ii_P_en3 = (er_P_en3*365*(value_of_Chla_3*1000000*depth))/1000000000000
    ii_P_en4 = (er_P_en4*365*(value_of_Chla_4*1000000*depth))/1000000000000
    return ii_N_dp1,ii_N_dp2,ii_N_dp3,ii_N_dp4,ii_N_en1,ii_N_en2,ii_N_en3,ii_N_en4,ii_P_dp1,ii_P_dp2,ii_P_dp3,ii_P_dp4,ii_P_en1,ii_P_en2,ii_P_en3,ii_P_en4
dq1,dq2,dq3,dq4 = Gusuan_Class(0.1,1)
tg_ALL_1 = Gusuan_TWO(539830,0.5,dq1,dq2,dq3,dq4)
dq1,dq2,dq3,dq4 = Gusuan_Class(1,10)
tg_ALL_2 = Gusuan_TWO(477540,5.4,dq1,dq2,dq3,dq4)
dq1,dq2,dq3,dq4 = Gusuan_Class(10,100)
tg_ALL_3 = Gusuan_TWO(423002,10.4,dq1,dq2,dq3,dq4)
dq1,dq2,dq3,dq4 = Gusuan_Class(100,1000)
tg_ALL_4 = Gusuan_TWO(1704431,63,dq1,dq2,dq3,dq4)

N_DP = tg_ALL_1[0] + tg_ALL_2[0] + tg_ALL_3[0] + tg_ALL_4[0]
P_DP = tg_ALL_1[1] + tg_ALL_2[1] + tg_ALL_3[1] + tg_ALL_4[1]
N_EN = tg_ALL_1[2] + tg_ALL_2[2] + tg_ALL_3[2] + tg_ALL_4[2]
P_EN = tg_ALL_1[3] + tg_ALL_2[3] + tg_ALL_3[3] + tg_ALL_4[3]

#print ('The estimated value [N_dp] : ' , N_DP )
#print ('The estimated value [P_DP] : ' , P_DP )
#print ('The estimated value [N_EN] : ' , N_EN )
#print ('The estimated value [P_EN] : ' , P_EN )

### Next

dq1,dq2,dq3,dq4 = Gusuan_Class(0.1,1)
tg_ALL_1 = Gusuan_plot(539830,0.5,dq1,dq2,dq3,dq4)
dq1,dq2,dq3,dq4 = Gusuan_Class(1,10)
tg_ALL_2 = Gusuan_plot(477540,5.4,dq1,dq2,dq3,dq4)
dq1,dq2,dq3,dq4 = Gusuan_Class(10,100)
tg_ALL_3 = Gusuan_plot(423002,10.4,dq1,dq2,dq3,dq4)
dq1,dq2,dq3,dq4 = Gusuan_Class(100,1000)
tg_ALL_4 = Gusuan_plot(1704431,63,dq1,dq2,dq3,dq4)
N_dp1 = tg_ALL_1[0] 
N_dp2 = tg_ALL_1[1] 
N_dp3 = tg_ALL_1[2] 
N_dp4 = tg_ALL_1[3] 
N_dp5 = tg_ALL_2[0] 
N_dp6 = tg_ALL_2[1] 
N_dp7 = tg_ALL_2[2] 
N_dp8 = tg_ALL_2[3] 
N_dp9 = tg_ALL_3[0] 
N_dp10 = tg_ALL_3[1] 
N_dp11 = tg_ALL_3[2] 
N_dp12 = tg_ALL_3[3] 
N_dp13 = tg_ALL_4[0] 
N_dp14 = tg_ALL_4[1] 
N_dp15 = tg_ALL_4[2] 
N_dp16 = tg_ALL_4[3]
N_en1 = tg_ALL_1[4] 
N_en2 = tg_ALL_1[5] 
N_en3 = tg_ALL_1[6] 
N_en4 = tg_ALL_1[7] 
N_en5 = tg_ALL_2[4] 
N_en6 = tg_ALL_2[5] 
N_en7 = tg_ALL_2[6] 
N_en8 = tg_ALL_2[7] 
N_en9 = tg_ALL_3[4] 
N_en10 = tg_ALL_3[5] 
N_en11 = tg_ALL_3[6] 
N_en12 = tg_ALL_3[7] 
N_en13 = tg_ALL_4[4] 
N_en14 = tg_ALL_4[5] 
N_en15 = tg_ALL_4[6] 
N_en16 = tg_ALL_4[7]
P_dp1 = tg_ALL_1[8] 
P_dp2 = tg_ALL_1[9] 
P_dp3 = tg_ALL_1[10] 
P_dp4 = tg_ALL_1[11] 
P_dp5 = tg_ALL_2[8] 
P_dp6 = tg_ALL_2[9] 
P_dp7 = tg_ALL_2[10] 
P_dp8 = tg_ALL_2[11] 
P_dp9 = tg_ALL_3[8] 
P_dp10 = tg_ALL_3[9] 
P_dp11 = tg_ALL_3[10] 
P_dp12 = tg_ALL_3[11] 
P_dp13 = tg_ALL_4[8] 
P_dp14 = tg_ALL_4[9] 
P_dp15 = tg_ALL_4[10] 
P_dp16 = tg_ALL_4[11]
P_en1 = tg_ALL_1[12] 
P_en2 = tg_ALL_1[13] 
P_en3 = tg_ALL_1[14] 
P_en4 = tg_ALL_1[15] 
P_en5 = tg_ALL_2[12] 
P_en6 = tg_ALL_2[13] 
P_en7 = tg_ALL_2[14] 
P_en8 = tg_ALL_2[15] 
P_en9 = tg_ALL_3[12] 
P_en10 = tg_ALL_3[13] 
P_en11 = tg_ALL_3[14] 
P_en12 = tg_ALL_3[15] 
P_en13 = tg_ALL_4[12] 
P_en14 = tg_ALL_4[13] 
P_en15 = tg_ALL_4[14] 
P_en16 = tg_ALL_4[15]

#%%  Space distribution
disemission2 = disemission.sort_values(by='Hylak_id')
d=disemission.iloc[456949,:]
disemissionchla2 = disemissionchla.sort_values(by='Hylak_id')
disemissionchla3 = disemissionchla2.drop_duplicates('Hylak_id')
merge_DATA = pd.merge(disemissionchla3,disemission,on='Hylak_id', how='inner')
def CCCC(s1,s2,yy1,yy2,leibie,mz):  ##g-N/day/m3
    merge_DATA_SZ1 = merge_DATA[merge_DATA['Lake_area'] >= s1 ]
    merge_DATA_SZ2 = merge_DATA_SZ1[merge_DATA_SZ1['Lake_area'] < s2]
    merge_DATA_SZ2_FY_1 =  merge_DATA_SZ2[merge_DATA_SZ2['chla'] >= yy1]
    merge_DATA_SZ2_FY_2 = merge_DATA_SZ2_FY_1[merge_DATA_SZ2_FY_1['chla'] < yy2]
    merge_DATA_SZ2_FY_2[mz] = leibie/len(merge_DATA_SZ2_FY_2 )
    gailei_1 = merge_DATA_SZ2_FY_2[['Hylak_id','Pour_long_x','Pour_lat_x',mz]].reset_index(drop=True)
    return gailei_1
n_dp1 = CCCC(0.1,1,0,2,N_dp1,'N_dp')
n_dp2 = CCCC(0.1,1,2,7,N_dp2,'N_dp')
n_dp3 = CCCC(0.1,1,7,30,N_dp3,'N_dp')
n_dp4 = CCCC(0.1,1,30,10000000,N_dp4,'N_dp')
n_dp5 = CCCC(1,10,0,2,N_dp5,'N_dp')
n_dp6 = CCCC(1,10,2,7,N_dp6,'N_dp')
n_dp7 = CCCC(1,10,7,30,N_dp7,'N_dp')
n_dp8 = CCCC(1,10,30,10000000,N_dp8,'N_dp')
n_dp9 = CCCC(10,100,0,2,N_dp9,'N_dp')
n_dp10 = CCCC(10,100,2,7,N_dp10,'N_dp')
n_dp11 = CCCC(10,100,7,30,N_dp11,'N_dp')
n_dp12 = CCCC(10,100,30,10000000,N_dp12,'N_dp')
n_dp13 = CCCC(100,100000,0,2,N_dp13,'N_dp')
n_dp14 = CCCC(100,100000,2,7,N_dp14,'N_dp')
n_dp15 = CCCC(100,100000,7,30,N_dp15,'N_dp')
n_dp16 = CCCC(100,100000,30,10000000,N_dp16,'N_dp')
n_dp_Aera = pd.concat([n_dp1,n_dp2,n_dp3,n_dp4,n_dp5,n_dp6,n_dp7,n_dp8,n_dp9,n_dp10,n_dp11,n_dp12,n_dp13,n_dp14,n_dp15,n_dp16]).reset_index(drop=True)
n_dp_Aera['N_dp_Kg'] = n_dp_Aera['N_dp'] * 1000000000
#np.sum(n_dp_Aera['N_dp'])

n_en1 = CCCC(0.1,1,0,2,N_en1,'N_en')
n_en2 = CCCC(0.1,1,2,7,N_en2,'N_en')
n_en3 = CCCC(0.1,1,7,30,N_en3,'N_en')
n_en4 = CCCC(0.1,1,30,10000000,N_en4,'N_en')
n_en5 = CCCC(1,10,0,2,N_en5,'N_en')
n_en6 = CCCC(1,10,2,7,N_en6,'N_en')
n_en7 = CCCC(1,10,7,30,N_en7,'N_en')
n_en8 = CCCC(1,10,30,10000000,N_en8,'N_en')
n_en9 = CCCC(10,100,0,2,N_en9,'N_en')
n_en10 = CCCC(10,100,2,7,N_en10,'N_en')
n_en11 = CCCC(10,100,7,30,N_en11,'N_en')
n_en12 = CCCC(10,100,30,10000000,N_en12,'N_en')
n_en13 = CCCC(100,100000,0,2,N_en13,'N_en')
n_en14 = CCCC(100,100000,2,7,N_en14,'N_en')
n_en15 = CCCC(100,100000,7,30,N_en15,'N_en')
n_en16 = CCCC(100,100000,30,10000000,N_en16,'N_en')
n_en_Aera = pd.concat([n_en1,n_en2,n_en3,n_en4,n_en5,n_en6,n_en7,n_en8,n_en9,n_en10,n_en11,n_en12,n_en13,n_en14,n_en15,n_en16]).reset_index(drop=True)
n_en_Aera['N_en_Kg'] = n_en_Aera['N_en'] * 1000000000
#np.sum(n_en_Aera['N_en'])

p_dp1 = CCCC(0.1,1,0,2,P_dp1,'P_dp')
p_dp2 = CCCC(0.1,1,2,7,P_dp2,'P_dp')
p_dp3 = CCCC(0.1,1,7,30,P_dp3,'P_dp')
p_dp4 = CCCC(0.1,1,30,10000000,P_dp4,'P_dp')
p_dp5 = CCCC(1,10,0,2,P_dp5,'P_dp')
p_dp6 = CCCC(1,10,2,7,P_dp6,'P_dp')
p_dp7 = CCCC(1,10,7,30,P_dp7,'P_dp')
p_dp8 = CCCC(1,10,30,10000000,P_dp8,'P_dp')
p_dp9 = CCCC(10,100,0,2,P_dp9,'P_dp')
p_dp10 = CCCC(10,100,2,7,P_dp10,'P_dp')
p_dp11 = CCCC(10,100,7,30,P_dp11,'P_dp')
p_dp12 = CCCC(10,100,30,10000000,P_dp12,'P_dp')
p_dp13 = CCCC(100,100000,0,2,P_dp13,'P_dp')
p_dp14 = CCCC(100,100000,2,7,P_dp14,'P_dp')
p_dp15 = CCCC(100,100000,7,30,P_dp15,'P_dp')
p_dp16 = CCCC(100,100000,30,10000000,P_dp16,'P_dp')
p_dp_Aera = pd.concat([p_dp1,p_dp2,p_dp3,p_dp4,p_dp5,p_dp6,p_dp7,p_dp8,p_dp9,p_dp10,p_dp11,p_dp12,p_dp13,p_dp14,p_dp15,p_dp16]).reset_index(drop=True)
p_dp_Aera['P_dp_Kg'] = p_dp_Aera['P_dp'] * 1000000000
#np.sum(p_dp_Aera['P_dp'])

p_en1 = CCCC(0.1,1,0,2,P_en1,'P_en')
p_en2 = CCCC(0.1,1,2,7,P_en2,'P_en')
p_en3 = CCCC(0.1,1,7,30,P_en3,'P_en')
p_en4 = CCCC(0.1,1,30,10000000,P_en4,'P_en')
p_en5 = CCCC(1,10,0,2,P_en5,'P_en')
p_en6 = CCCC(1,10,2,7,P_en6,'P_en')
p_en7 = CCCC(1,10,7,30,P_en7,'P_en')
p_en8 = CCCC(1,10,30,10000000,P_en8,'P_en')
p_en9 = CCCC(10,100,0,2,P_en9,'P_en')
p_en10 = CCCC(10,100,2,7,P_en10,'P_en')
p_en11 = CCCC(10,100,7,30,P_en11,'P_en')
p_en12 = CCCC(10,100,30,10000000,P_en12,'P_en')
p_en13 = CCCC(100,100000,0,2,P_en13,'P_en')
p_en14 = CCCC(100,100000,2,7,P_en14,'P_en')
p_en15 = CCCC(100,100000,7,30,P_en15,'P_en')
p_en16 = CCCC(100,100000,30,10000000,P_en16,'P_en')
p_en_Aera = pd.concat([p_en1,p_en2,p_en3,p_en4,p_en5,p_en6,p_en7,p_en8,p_en9,p_en10,p_en11,p_en12,p_en13,p_en14,p_en15,p_en16]).reset_index(drop=True)
p_en_Aera['P_en_Kg'] = p_en_Aera['P_en'] * 1000000000
#np.sum(p_en_Aera['P_en'])


#n_dp_Aera.to_csv('N_dp(kg).csv',index=False)
#n_en_Aera.to_csv('N_en(kg).csv',index=False)
#p_dp_Aera.to_csv('P_dp(kg).csv',index=False)
#p_en_Aera.to_csv('P_en(kg).csv',index=False)

#%% ln(En_N/De_N) - ln(En_P/De_P) retention preference， >0 - N preference，<0 - P preference
e_N_DE = n_dp_Aera.iloc[:,-1].values
e_N_EN = n_en_Aera.iloc[:,-1].values
e_P_DE = p_dp_Aera.iloc[:,-1].values
e_P_EN = p_en_Aera.iloc[:,-1].values
rejust_e_result = pd.DataFrame(e_N_EN/e_N_DE).apply(np.log)  -  pd.DataFrame(e_P_EN/e_P_DE).apply(np.log)
rejust_e_result['Hylak_id'] = p_en_Aera['Hylak_id']
rejust_e_result.columns = ['VV','Hylak_id']
rejust_e_result2 = rejust_e_result.drop_duplicates(subset=['Hylak_id'])
merge_rejust = pd.merge(upscaling_database.reset_index(drop=True),rejust_e_result2.reset_index(drop=True),on='Hylak_id',how='outer')
#merge_rejust.to_csv('V_RESULRT_function_poly.csv',index=False)


#%% Spatial cross correlation

min_max_scaler = preprocessing.MinMaxScaler() 
filepathQE = filepathQ + '/Corr/RR/'

base = importr('base')    #python import R__base
library1 = importr('spatialEco')
library2 = importr('sp')
library3 = importr('spdep')



p_data = pd.read_csv(filepathQE + 'P_varial.csv')
## if change 'P_varial.csv' to 'N_varial'  Then the result of N will be change too

qqc = 0
dd = 0
aerson_coor = []
bJHXG = []
while dd <= len(p_data):
    qq1 = p_data.iloc[qqc*10 : (qqc + 1 ) * 10]
    qqc += 1
    dd =     (qqc + 1 ) * 10
    #qqc = 101
    try:
        qq2 = qq1.drop_duplicates()
        qq2.to_csv(filepathQE + 'TT_PP.csv',index=False)  ## if change 'TT_PP' to 'TT_NN'  Then the result of N will be change too
        songxue1 = robjects.r("songxue <- read.csv(filepathQE + 'TT_PP.csv')")  ## if change 'TT_PP' to 'TT_NN'  Then the result of N will be change too
        songxue2 = robjects.r('coordinates(songxue) <- ~x+y  ')
        songxue3 = robjects.r('(I <- crossCorrelation(songxue$x, songxue$y, coords = coordinates(songxue), k=99 ) )')
        songxue4 = robjects.r('cc = I$SCI$lsci.xy')
        songxue5 = robjects.r("write.csv(cc,file=filepathQE + 'p_rec.csv')")   ## if change 'p_rec.csv' to 'n_rec.csv'  Then the result of N will be change too
        adds = pd.read_csv( filepathQE + 'p_rec.csv') ## if change 'p_rec.csv' to 'n_rec.csv'  Then the result of N will be change too
        adds_C = np.median(adds['x'])
        adds_Mean = np.mean(adds['x'])
       # print (adds_C )
        #print (adds_Mean )
        p_valur =  list ([np.round(pearsonr(qq1.iloc[:,0],qq1.iloc[:,1])[0],3) ] * 10)
        aerson_coor.append(  p_valur   )
        jjhxg = list ([ np.round(adds_Mean,6)  ] * 10)
        bJHXG.append(jjhxg)
    except : 
        p_valur =  list ([ 0 ] * 10)
        aerson_coor.append(  p_valur   )
        jjhxg = list ([ 0 ] * 10)
        bJHXG.append(jjhxg)

mu = pd.DataFrame(aerson_coor)  #R
mu2 = mu.stack()
kio = pd.DataFrame(bJHXG)   # Indirect correlation
kio2 = kio.stack()
zjxg = mu2- kio2  # direct correlation
aMDU = pd.concat([mu2,kio2,zjxg],axis=1)
#%%

<font face="Times New Roman" >
<font size=3 >
The calculation results are as follows (need to import the following |pyc| file, which can be found in dir_code)

In [1]:
import sys
import pandas as pd
sys.path.append(r'C:\Users\JL\Desktop\Globallake_File\__pycache__') 
import Upscaling_pyc

+------------------+-------------------+-------------------+------------------+----------------------+
| Chla class(ug/L) | <2 (Oligotrophic) | 2-7 (Mesotrophic) | 7-30 (Eutrophic) | >30 (Hypereutrophic) |
+------------------+-------------------+-------------------+------------------+----------------------+
|       N_DP       |  0.94(0.65,1.23)  |  3.63(2.40,4.87)  | 4.03(1.33,6.74)  |        23.14         |
|       P_DP       |  0.27(0.21,0.33)  |  1.04(0.56,1.52)  | 1.34(0.25,2.44)  |         6.10         |
|       N_EN       |  1.31(1.05,1.57)  |  1.44(1.01,1.88)  | 0.21(0.03,0.39)  |         0.21         |
|       P_EN       |  0.24(0.19,0.29)  |  0.27(0.18,0.36)  | 0.04(0.01,0.08)  |         0.03         |
+------------------+-------------------+-------------------+------------------+----------------------+
